# PréProcessamento dos dados

Fonte de Dados sobre PréProcessamento

````
Livro Python Machine Learning de Sebastian Raschka: o capítulo 4 do livro é sobre a construção de boas bases de dados utilizando técnicas de pré-processamento

Livro Machine Learning with Python de Chris Albon: os capítulos iniciais do livro apresentam uma variedade grande de técnicas de pré-processamento

Capítulo Data Quality de Tamraparni Dasu e Theodore Johnson da série de livros Wiley Series in Probability and Statistics: ótimo complemento depois da aprendizagem das técnicas de pré-processamento
````

In [1]:
import pandas as pd
import numpy as np

## Exemplo da Base de Dados de Crétido

## Descrevendo a base de crédito

Base com 2000 clientes, um registro para cada.

Features
+ `clientid`: Um ID para o cliente
    - Variável Categórica Nominal
+ `income`: Salário/Renda da pessoa no formato Anual
    - Variável Numérica Contínua
+ `age`: Idade da pessoa
    - Variável Numérica Contínua ou pode ser convertida para Discreta
+ `loan`: Quantidade de empréstimo realizado
    - Variável Numérica Contínua
Classe a ser Prevista
+ `default`: 0: Não foi capaz de pagar o empréstimo || 1: Foi capas de pagar


In [2]:
base = pd.read_csv('files/credit-data.csv')

In [3]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
clientid    2000 non-null int64
income      2000 non-null float64
age         1997 non-null float64
loan        2000 non-null float64
default     2000 non-null int64
dtypes: float64(3), int64(2)
memory usage: 78.2 KB


In [4]:
base.describe()

,clientid,income,age,loan,default
count,2000.000000,2000.000000,1997.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.807559,4444.369695,0.141500
std,577.494589,14326.327119,13.624469,3045.410024,0.348624
min,1.000000,20014.489470,-52.423280,1.377630,0.000000
25%,500.750000,32796.459717,28.990415,1939.708847,0.000000
50%,1000.500000,45789.117313,41.317159,3974.719419,0.000000
75%,1500.250000,57791.281668,52.587040,6432.410625,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000


In [6]:
base.head()

,clientid,income,age,loan,default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1


### Handle Valores Inconsistente

Perceba que na idade, o `min()` dela é negativo, e não existe idade negativo. Então, isso deve ser corrigido

In [7]:
base.describe()

,clientid,income,age,loan,default
count,2000.000000,2000.000000,1997.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.807559,4444.369695,0.141500
std,577.494589,14326.327119,13.624469,3045.410024,0.348624
min,1.000000,20014.489470,-52.423280,1.377630,0.000000
25%,500.750000,32796.459717,28.990415,1939.708847,0.000000
50%,1000.500000,45789.117313,41.317159,3974.719419,0.000000
75%,1500.250000,57791.281668,52.587040,6432.410625,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000


In [8]:
# Pegando os cliente com 'age' negativa
base.loc[base['age'] < 0]

,clientid,income,age,loan,default
15,16,50501.726689,-28.218361,3977.287432,0
21,22,32197.620701,-52.423280,4244.057136,0
26,27,63287.038908,-36.496976,9595.286289,0


**O que fazer com valores inconsistêntes**
````python

# 1. apagar toda a coluna (nesse caso, isso é ruim)
base.drop('age', 1, inplace=True)

# 2. apagar somente os registros com problema
base.drop(base[base.age < 0].index, inplace=True)

# 3. preencher os valores manualmente (cada um)
## Mas nesse caso é impssível

# 4. preencher os valores com a média (esse é o mais idela e VAMOS USAR)
base.mean()
base['age'].mean()
base['age'][base.age > 0].mean()
base.loc[base.age < 0, 'age'] = 40.92
````

In [9]:
# No caso, vamos apliacr a opçâo 4: Preencher com a média
media = base['age'][base.age > 0].mean() # A média nâo pdoe incluir os valores inválidos
print("media", media)
base.loc[base.age < 0, 'age'] = media


media 40.92770044906149


In [10]:
# Verificando se tem valores invaldiso
base.loc[base['age'] < 0]
# Não há mais dados inválidos

,clientid,income,age,loan,default


### Valores Faltantes

O que fazer quando há dados inválidos. Dados faltantes devem ser corrigidos pois nos algoritmo de ML pode dar problema.

In [11]:
base.loc[pd.isnull(base['age'])]

,clientid,income,age,loan,default
28,29,59417.805406,NaN,2082.625938,0
30,31,48528.852796,NaN,6155.784670,0
31,32,23526.302555,NaN,2862.010139,0


Vamos separar em base de features e de tags

In [12]:
# Essa separação vai converter para numpy
previsores = base.iloc[:, 1:4].values
classe = base.iloc[:, 4].values

In [13]:
# Dados agora expressados em numpy
previsores

array([[6.61559251e+04, 5.90170151e+01, 8.10653213e+03],
       [3.44151540e+04, 4.81171531e+01, 6.56474502e+03],
       [5.73171701e+04, 6.31080495e+01, 8.02095330e+03],
       ...,
       [4.43114493e+04, 2.80171669e+01, 5.52278669e+03],
       [4.37560566e+04, 6.39717958e+01, 1.62272260e+03],
       [6.94365796e+04, 5.61526170e+01, 7.37883360e+03]])

In [14]:
# Substituindo valores faltantes NaN pela média
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(previsores[:, 0:3])
previsores[:, 0:3] = imputer.transform(previsores[:,0:3])

### Escalonamento dos Atributos

Os atributos tem escalas diferentes, nesse exemplo, a idade varia entre 1 e 90 enquanto que a renda anual e o empréstimo entre 0 a 100.000.


Por conta disso, certos algorimos podem favorecer os atributos com maior valor escalar do que o que tem menos, sendo que todos tem a mesma importâcia. Para que isso não aconteceça, temos que normalizar

Então é necessário fazer um escalonamento. Assim deixaremos a base de forma que não haja difenreça entre a renda e a idade.

Além disso, o escalonamento deixa o algoritmo mais rápido por reduzir seu tamanho escalar dos dados (de 1000 para \[0,1\])

![img02-standardisation](imgs/img02-standardisation.png)

In [ ]:
# Valores antes do escalonamento
previsores

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

In [16]:
# Valores depois do escalonamento
previsores

array([[ 1.45393393,  1.36538005,  1.20281942],
       [-0.76217555,  0.54265932,  0.69642695],
       [ 0.83682073,  1.67417101,  1.17471147],
       ...,
       [-0.07122592, -0.97448606,  0.35420081],
       [-0.11000289,  1.73936652, -0.92675625],
       [ 1.682986  ,  1.14917551,  0.96381038]])

## Base do Census

https://archive.ics.uci.edu/ml/datasets/Census+Income

Baseado nas Features, prever se tem um salário de >50k ou <50k

In [17]:
base_census = pd.read_csv('files/census.csv')

In [18]:
base_census.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
final-weight      32561 non-null int64
education         32561 non-null object
education-num     32561 non-null int64
marital-status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital-gain      32561 non-null int64
capital-loos      32561 non-null int64
hour-per-week     32561 non-null int64
native-country    32561 non-null object
income            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [19]:
base_census.describe()

,age,final-weight,education-num,capital-gain,capital-loos,hour-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [20]:
base_census.head(2)

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K


### Tranformar Variáveis Categóricas em Numéricas

In [21]:
## Separar features das tags
previsores_census = base_census.iloc[:, 0:14].values
classe_census = base_census.iloc[:, 14].values

In [22]:
# Como são as features antes da conversão
previsores_census[0]

array([39, ' State-gov', 77516, ' Bachelors', 13, ' Never-married',
       ' Adm-clerical', ' Not-in-family', ' White', ' Male', 2174, 0, 40,
       ' United-States'], dtype=object)

#### Variável Dummy : Para variáveis Categóricas Nominais

OneHotCode: Converter cada valor categórico difente em uma coluna que recebe 0/1 para aquele determinado valor.

Isso implica em criara mais colunas

![img01-variable-dummy](imgs/img01-variable-dummy.png)

In [24]:
# Nesse caso, vamos tranformar toda a base de uma só vez
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [1,3,5,6,7,8,9,13])],remainder='passthrough')
previsores_census = onehotencorder.fit_transform(previsores_census).toarray()

In [25]:
# features depois da conversão
previsores_census[0]

array([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e

LabelEncoder

Serve para converter as classe de "<=50k" e ">50k" para 0 e 1

In [26]:
# tags antes da conversão do LabelEncoder
classe_census[0]

' <=50K'

In [27]:
# labelenconder: converte os dadaos categoricos em núemro diretos
# converte " >=50" para 0 e ">50" para 1
from sklearn.preprocessing import LabelEncoder

labelencorder_classe = LabelEncoder()
classe_census = labelencorder_classe.fit_transform(classe_census)

In [28]:
# tag depois da conversão do LabelEncoder
classe_census[0]

0

### Escalonamento das `features` na base `census`

In [29]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

# Divisão das Bases em `train` e `test`

## Divisão da Base de Crédito

In [33]:
# Função para dividir a base em base de testes e de treino
from sklearn.model_selection import train_test_split 
# Estamos dividino em 75% para treinar e 25% para testar. 
# random_state = 0
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(
    previsores, classe, test_size = 0.25, random_state = 0)

In [35]:
previsores_treinamento.shape

(1500, 3)

In [36]:
previsores_teste.shape

(500, 3)

## Divisão da Bases de `census`

In [37]:
from sklearn.model_selection import train_test_split 
previsores_treinamento_census, previsores_teste_census, classe_treinamento_census, classe_teste_census = train_test_split(
    previsores_census, classe_census, test_size = 0.25, random_state = 0)

In [38]:
previsores_treinamento_census.shape

(24420, 108)

In [39]:
previsores_teste_census.shape

(8141, 108)